In [ ]:
from fashionscrapper.utils.list import flatten, distinct

import numpy as np
from tqdm.auto import tqdm
from fashiondatasets.deepfashion2.helper.pairs.similar_embeddings import calculate_most_similar
from fashiondatasets.deepfashion1.helper.cbir_helper import flatten_distinct_values
from fashiondatasets.utils.list import parallel_map
from fashiondatasets.deepfashion1.DeepFashion1CBIR import DeepFashion1CBIR
from fashiondatasets.deepfashion1.helper.notebook import distances_existence_filtered, load_results, calc_top_k_from_embeddings
import numpy as np
import pickle

base_path = r"F:\workspace\datasets\deep_fashion_1_256"
emb_base = r"C:\workspace"

In [ ]:
jobs = distances_existence_filtered(emb_base, False)
embedding_path = jobs[1][0]
embedding_path

In [ ]:
r = calc_top_k_from_embeddings(base_path, embedding_path)

In [ ]:
jobs

In [ ]:
results = load_results(emb_base, verbose=True)

In [ ]:
results[0].keys()

In [ ]:
data = [{k: r[k] for k in [k for k in r.keys() if k != "result"]} for r in results]
import pandas as pd 
pd.DataFrame(data)


#embedding_path

In [ ]:
def clean_result(r):
    return r["query"], r["distances"]["matches"]
    

In [ ]:
from collections import defaultdict
data = defaultdict(lambda: [])

for r in results:
    for l in r["result"]:
        q, matches = clean_result(l)
        data[q].extend(matches)

In [ ]:
data_distances = {}
data_dist_idxs = {}
for q, matches in data.items():
    distances = min([x[1] for x in matches])
    idxs = min([x[2] for x in matches])    
    
    data_distances[q] = (distances, matches)
    data_dist_idxs[q] = (idxs, matches)
    
data_distances = {k: v for k, v in reversed(sorted(data_distances.items(), key=lambda item: item[1][0]))}
data_dist_idxs = {k: v for k, v in reversed(sorted(data_dist_idxs.items(), key=lambda item: item[1][0]))}

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt

In [ ]:
import os
f_path = lambda f: os.path.join(r"F:\workspace\datasets\deep_fashion_1_256\img", f)
start = 50
__data = list(data_distances.items())[start:start+25]
for q, (dist, matches) in __data:
    matches = distinct([f_path(m) for m, dist, idx in matches])

    a = Image.open(f_path(q))    
    fig = plt.figure()
    
    ax1 = fig.add_subplot(2,2,1)    
    ax1.imshow(a)
    ax1.title.set_text(q.replace(r"F:\workspace\datasets\deep_fashion_1_256\img\img\img", "").replace("img/", ""))
    plt.axis('off')
    
    for m in matches:
        p = Image.open(m)
        
        ax2 = fig.add_subplot(2,2,2)
        ax2.imshow(p)
        #ax2.title.set_text(m.replace(r"F:\workspace\datasets\deep_fashion_1_256\img", "").replace("img/", ""))
        plt.axis('off')
        plt.xlabel(q)